# The CORD-19 Dataset

CORD-19 is a collection of over 50,000 scholarly articles - including over 40,000 with full text - about COVID-19, SARS-CoV-2, and related coronaviruses. This dataset has been made freely available with the goal to aid research communities combat the COVID-19 pandemic.

The goal of this notebook is two-fold:

1. Demonstrate how to access the CORD-19 dataset on Azure: We connect to the Azure blob storage account housing the CORD-19 dataset.
2. Walk though the structure of the dataset: Articles in the dataset are stored as json files. We provide examples showing:

  - How to find the articles (navigating the container)
  - How to read the articles (navigating the json schema)

Dependencies: This notebook requires the following libraries:

- Azure storage (e.g. `pip install azure-storage`)
- NLTK ([docs](https://www.nltk.org/install.html))
- Pandas (e.g. `pip install pandas`)

## Getting the CORD-19 data from Azure

The CORD-19 data has been uploaded as an Azure Open Dataset [here](https://azure.microsoft.com/services/open-datasets/catalog/covid-19-open-research/). We create a blob service linked to this CORD-19 open dataset.

In [1]:
from azure.storage.blob import BlockBlobService

# storage account details
azure_storage_account_name = "azureopendatastorage"
azure_storage_sas_token = "sv=2019-02-02&ss=bfqt&srt=sco&sp=rlcup&se=2025-04-14T00:21:16Z&st=2020-04-13T16:21:16Z&spr=https&sig=JgwLYbdGruHxRYTpr5dxfJqobKbhGap8WUtKFadcivQ%3D"

# create a blob service
blob_service = BlockBlobService(
    account_name=azure_storage_account_name,
    sas_token=azure_storage_sas_token,
)

We can use this blob service as a handle on the data. We can navigate the dataset making use of the `BlockBlobService` APIs. See here for more details:

* [Blob service concepts](https://docs.microsoft.com/en-us/rest/api/storageservices/blob-service-concepts)
* [Operations on containers](https://docs.microsoft.com/en-us/rest/api/storageservices/operations-on-containers)

The CORD-19 data is stored in the `covid19temp` container. This is the file structure within the container together with an example file.

```
metadata.csv
custom_license/
    pdf_json/
        0001418189999fea7f7cbe3e82703d71c85a6fe5.json        # filename is sha-hash
        ...
    pmc_json/
        PMC1065028.xml.json                                  # filename is the PMC ID
        ...
noncomm_use_subset/
    pdf_json/
        0036b28fddf7e93da0970303672934ea2f9944e7.json
        ...
    pmc_json/
        PMC1616946.xml.json
        ...
comm_use_subset/
    pdf_json/
        000b7d1517ceebb34e1e3e817695b6de03e2fa78.json
        ...
    pmc_json/
        PMC1054884.xml.json
        ...
biorxiv_medrxiv/                                             # note: there is no pmc_json subdir
    pdf_json/
        0015023cc06b5362d332b3baf348d11567ca2fbb.json
        ...
```

Each .json file corresponds to an individual article in the dataset. This is where the title, authors, abstract and (where available) the full text data is stored.

## Using metadata.csv
The CORD-19 dataset comes with `metadata.csv` - a single file that records basic information on all the papers available in the CORD-19 dataset. This is a good place to start exploring!

In [2]:
# container housing CORD-19 data
container_name = "covid19temp"

# download metadata.csv
metadata_filename = 'metadata.csv'
blob_service.get_blob_to_path(
    container_name=container_name,
    blob_name=metadata_filename,
    file_path=metadata_filename
)

In [3]:
import pandas as pd

# read metadata.csv into a dataframe
metadata_filename = 'metadata.csv'
metadata = pd.read_csv(metadata_filename)

In [4]:
metadata.head(3)

cord_uid                                       sha source_x  \
0  xqhn0vbp  1e1286db212100993d03cc22374b624f7caee956      PMC   
1  gi6uaa83  8ae137c8da1607b3a8e4c946c07ca8bda67f88ac      PMC   
2  le0ogx1s                                       NaN      PMC   

                                               title                      doi  \
0  Airborne rhinovirus detection and effect of ul...    10.1186/1471-2458-3-5   
1    Discovering human history from stomach bacteria  10.1186/gb-2003-4-5-213   
2     A new recruit for the army of the men of death  10.1186/gb-2003-4-7-113   

       pmcid   pubmed_id license  \
0  PMC140314  12525263.0   no-cc   
1  PMC156578  12734001.0   no-cc   
2  PMC193621  12844350.0   no-cc   

                                            abstract publish_time  \
0  BACKGROUND: Rhinovirus, the most common cause ...   2003-01-13   
1  Recent analyses of human pathogens have reveal...   2003-04-28   
2  The army of the men of death, in John Bunyan's...   2003-06-27   

                                             authors            journal  \
0  Myatt, Theodore A; Johnston, Sebastian L; Rudn...  BMC Public Health   
1                                   Disotell, Todd R        Genome Biol   
2                                  Petsko, Gregory A        Genome Biol   

   Microsoft Academic Paper ID WHO #Covidence  has_pdf_parse  \
0                          NaN            NaN           True   
1                          NaN            NaN           True   
2                          NaN            NaN          False   

   has_pmc_xml_parse  full_text_file  \
0               True  custom_license   
1               True  custom_license   
2               True  custom_license   

                                                 url  
0  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...  
1  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...  
2  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...

That's a lot to take in at first glance, so let's apply a little polish.

In [5]:
simple_schema = ['cord_uid', 'source_x', 'title', 'abstract', 'authors', 'full_text_file', 'url']

def make_clickable(address):
    '''Make the url clickable'''
    return '<a href="{0}">{0}</a>'.format(address)

def preview(text):
    '''Show only a preview of the text data.'''
    return text[:30] + '...'

format_ = {'title': preview, 'abstract': preview, 'authors': preview, 'url': make_clickable}

metadata[simple_schema].head().style.format(format_)

In [6]:
# let's take a quick look around
num_entries = len(metadata)
print("There are {} many entries in this dataset:".format(num_entries))

metadata_with_text = metadata[metadata['full_text_file'].isna() == False]
with_full_text = len(metadata_with_text)
print("-- {} have full text entries".format(with_full_text))

with_doi = metadata['doi'].count()
print("-- {} have DOIs".format(with_doi))

with_pmcid = metadata['pmcid'].count()
print("-- {} have PubMed Central (PMC) ids".format(with_pmcid))

with_microsoft_id = metadata['Microsoft Academic Paper ID'].count()
print("-- {} have Microsoft Academic paper ids".format(with_microsoft_id))

There are 51078 many entries in this dataset:
-- 42511 have full text entries
-- 47741 have DOIs
-- 41082 have PubMed Central (PMC) ids
-- 964 have Microsoft Academic paper ids


## Example: Read full text

Notice that `metadata.csv` does not contain the full-text itself. Let's see an example of how to read that. We will locate and unpack the full text json and convert it to a list of sentences.

In [7]:
# choose a random example with pdf parse available
metadata_with_pdf_parse = metadata[metadata['has_pdf_parse']]
example_entry = metadata_with_pdf_parse.iloc[42]

# construct path to blob containing full text
blob_name = '{0}/pdf_json/{1}.json'.format(example_entry['full_text_file'], example_entry['sha'])  # note the repetition in the path
print("Full text blob for this entry:")
print(blob_name)

Full text blob for this entry:
custom_license/pdf_json/f1d1b9694aa43c837d9b758cb2d45d8a24d293e3.json


We can now read the json content associated to this blob as follows.

In [8]:
import json
blob_as_json_string = blob_service.get_blob_to_text(container_name=container_name, blob_name=blob_name)
data = json.loads(blob_as_json_string.content)

# in addition to the body text, the metadata is also stored within the individual json files
print("Keys within data:", ', '.join(data.keys()))

Keys within data: paper_id, metadata, abstract, body_text, bib_entries, ref_entries, back_matter


For the purposes of this example we are interested in the `body_text`, which stores the text data as follows:

```json
"body_text": [                      # list of paragraphs in full body
    {
        "text": <str>,
        "cite_spans": [             # list of character indices of inline citations
                                    # e.g. citation "[7]" occurs at positions 151-154 in "text"
                                    #      linked to bibliography entry BIBREF3
            {
                "start": 151,
                "end": 154,
                "text": "[7]",
                "ref_id": "BIBREF3"
            },
            ...
        ],
        "ref_spans": <list of dicts similar to cite_spans>,     # e.g. inline reference to "Table 1"
        "section": "Abstract"
    },
    ...
]
```

The full json schema is available [here](https://ai2-semanticscholar-cord-19.s3-us-west-2.amazonaws.com/2020-03-13/json_schema.txt).

In [9]:
from nltk.tokenize import sent_tokenize

# the text itself lives under 'body_text'
text = data['body_text']

# many NLP tasks play nicely with a list of sentences
sentences = []
for paragraph in text:
    sentences.extend(sent_tokenize(paragraph['text']))

print("An example sentence:", sentences[0])

An example sentence: It is now widely admitted that actual genomes have a common ancestor (LUCA, Last Universal Common Ancestor).


### PDF vs PMC XML Parse

In the above example we looked at a case with `has_pdf_parse == True`. In that case the blob file path was of the form:

```
'<full_text_file>/pdf_json/<sha>.json'
```

Alternatively, for cases with `has_pmc_xml_parse == True` use the following format:

```
'<full_text_file>/pmc_json/<pmcid>.xml.json'
```

For example:

In [10]:
# choose a random example with pmc parse available
metadata_with_pmc_parse = metadata[metadata['has_pmc_xml_parse']]
example_entry = metadata_with_pmc_parse.iloc[42]

# construct path to blob containing full text
blob_name = '{0}/pmc_json/{1}.xml.json'.format(example_entry['full_text_file'], example_entry['pmcid'])  # note the repetition in the path
print("Full text blob for this entry:")
print(blob_name)

blob_as_json_string = blob_service.get_blob_to_text(container_name=container_name, blob_name=blob_name)
data = json.loads(blob_as_json_string.content)

# the text itself lives under 'body_text'
text = data['body_text']

# many NLP tasks play nicely with a list of sentences
sentences = []
for paragraph in text:
    sentences.extend(sent_tokenize(paragraph['text']))

print("An example sentence:", sentences[0])

Full text blob for this entry:
custom_license/pmc_json/PMC546170.xml.json
An example sentence: Double-stranded small interfering RNA (siRNA) molecules have drawn much attention since it was unambiguously shown that they mediate potent gene knock-down in a variety of mammalian cells (1).


## Iterate through blobs directly

In the above examples we used the `metadata.csv` file to navigate the data, construct the blob file path, and read data from the blob. An alternative is the iterate through the blobs themselves.

In [11]:
# get and sort list of available blobs
blobs = blob_service.list_blobs(container_name)
sorted_blobs = sorted(list(blobs), key=lambda e: e.name, reverse=True)

Now we can iterate through the blobs directly. For example, let's count the number json files available.

In [12]:
# we can now iterate directly though the blobs
count = 0
for blob in sorted_blobs:
    if blob.name[-5:] == ".json":
        count += 1
print("There are {} many json files".format(count))

There are 59784 many json files


# Appendix

## Data quality issues

This is a large dataset that for obvious reasons was put together rather hastily! Here are some data quality issues's we've observed.

### Multiple shas

We observe that in some cases there are multiple shas for a given entry.

In [13]:
metadata_multiple_shas = metadata[metadata['sha'].str.len() > 40]

print("There are {} many entries with multiple shas".format(len(metadata_multiple_shas)))

metadata_multiple_shas.head(3)

There are 1999 many entries with multiple shas


cord_uid                                                sha source_x  \
20   fpj5urao  e9c78584c08ba79d735e150eff98297eb57f12dd; cdb2...      PMC   
93   2vlvz5o9  bd92cbae7179f07d59d1ce4d7ca96e37ebb40ec9; 7526...      PMC   
181  2g4m0dy7  2bd6e33d92632dfcba4056a2d7355ced5b7ab1fd; 7648...      PMC   

                                                 title  \
20   Moderate mutation rate in the SARS coronavirus...   
93   Design of Wide-Spectrum Inhibitors Targeting C...   
181  Reducing the Impact of the Next Influenza Pand...   

                              doi       pmcid   pubmed_id license  \
20         10.1186/1471-2148-4-21   PMC446188  15222897.0   no-cc   
93   10.1371/journal.pbio.0030324  PMC1197287  16128623.0   cc-by   
181  10.1371/journal.pmed.0030361  PMC1526768  16881729.0   cc-by   

                                              abstract publish_time  \
20   BACKGROUND: The outbreak of severe acute respi...   2004-06-28   
93   The genus Coronavirus contains about 25 specie...   2005-09-06   
181  BACKGROUND: The outbreak of highly pathogenic ...   2006-08-08   

                                               authors        journal  \
20   Zhao, Zhongming; Li, Haipeng; Wu, Xiaozhuang; ...  BMC Evol Biol   
93   Yang, Haitao; Xie, Weiqing; Xue, Xiaoyu; Yang,...      PLoS Biol   
181  Wu, Joseph T; Riley, Steven; Fraser, Christoph...       PLoS Med   

     Microsoft Academic Paper ID WHO #Covidence  has_pdf_parse  \
20                           NaN            NaN           True   
93                           NaN            NaN           True   
181                          NaN            NaN           True   

     has_pmc_xml_parse   full_text_file  \
20                True   custom_license   
93                True  comm_use_subset   
181               True  comm_use_subset   

                                                   url  
20   https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...  
93   https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...  
181  https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...

## Layout of the container

Here we use a simple regex to explore the file structure of the container in case this is updated in the future.

In [14]:
container_name = "covid19temp"
blobs = blob_service.list_blobs(container_name)
sorted_blobs = sorted(list(blobs), key=lambda e: e.name, reverse=True)

In [15]:
import re
dirs = {}

pattern = '([\w]+)\/([\w]+)\/([\w.]+).json'
for blob in sorted_blobs:
    
    m = re.match(pattern, blob.name)
    
    if m:
        dir_ = m[1] + '/' + m[2]
        
        if dir_ in dirs:
            dirs[dir_] += 1
        else:
            dirs[dir_] = 1
        
dirs

{'noncomm_use_subset/pmc_json': 2221,
 'noncomm_use_subset/pdf_json': 2507,
 'custom_license/pmc_json': 7854,
 'custom_license/pdf_json': 26799,
 'comm_use_subset/pmc_json': 9169,
 'comm_use_subset/pdf_json': 9539,
 'biorxiv_medrxiv/pdf_json': 1695}